# Dataset Similarity

The analyzing the similar datasets offers several use case for our data management purposes. Such as dataset recommendation and data integration or dataset merging. There are four dimensions of similarity between datasets which are be considered:

### 1.	Spatial Similarity:
The spatial similarity between two datasets is determined by calculating the geospatial area they cover. As is commonly observed, the longitude and latitude values for the same location may not match exactly when recorded from different sources. To account for this, hexagonal grids (with a 20-kilometer side length) are created. Points (longitude and latitude) that fall within the same grid are considered to cover the same area. The spatial similarity between Dataset #A and Dataset #B is expressed as the percentage of grids covered by Dataset #A, divided by the total number of grids covered by Dataset #B. Thus, this score is asymmetric in nature.

### 2.	Spatial Similarity:
The spatial similarity between two datasets is determined by calculating the geospatial area they cover. As is commonly observed, the longitude and latitude values for the same location may not match exactly when recorded from different sources. To account for this, hexagonal grids (with a 20-kilometer side length) are created. Points (longitude and latitude) that fall within the same grid are considered to cover the same area. The spatial similarity between Dataset #A and Dataset #B is expressed as the percentage of grids covered by Dataset #A, divided by the total number of grids covered by Dataset #B. Thus, this score is asymmetric in nature.

### 3.	Attribute Similarity:
The attributes of dataset signifies the type of dataset either capture biochemical or ocean variables. Similar attributes between two datasets may interest user to look into similar data. The attribute similarity is calculated as percentage of same attributes in Dataset#A with respect to Dataset#B. Thus, this score is asymmetric in nature.




## Step#1: Downloading Metadata from Erddap

This step will traverse all the data available on CIOOS Altantic and saves `alldataset_das_dict_NEW.pkl`


In [ ]:
import pickle
import time
from requests_html import HTMLSession
from tqdm import tqdm
import json

 # base_url = "https://cioosatlantic.ca/erddap/tabledap/index.html?page=1&itemsPerPage=1000"
base_url  = "https://catalogue.cioosatlantic.ca/api/3/action/resource_search?query=name:erddap"

def parse_json(json_object_):
    ds_profile_dict = {}
    lst_of_rows_ = json_object_['table']['rows']

    last_ = None
    for rtype, v_name, att_name, dtype, val_  in lst_of_rows_:
        if rtype == "variable":
            if v_name not in ds_profile_dict:
                ds_profile_dict[v_name] = {"dtype": dtype, "attr": {}}
            last_ = (rtype, v_name, att_name, dtype, val_)
        elif rtype == 'attribute':
            if v_name not in ds_profile_dict:
                ds_profile_dict[v_name] = {}
            else:
                if (last_ is not None) and last_[0] == 'variable':
                    assert att_name not in ds_profile_dict[v_name]["attr"], "error unknown"
                    ds_profile_dict[v_name]["attr"][att_name] = ( dtype, val_)
                else:
                    ds_profile_dict[v_name][att_name] = (dtype, val_)
        else:
            print("=====> Row type Error (Not found)")
            exit(-1)

    return ds_profile_dict

session = HTMLSession()
# Send an HTTP GET request to the URL
response = session.get(base_url)
# Render JavaScript if needed (optional)
# response.html.render()
json_list_links = json.loads(response.text)
all_elements = [r_id['url'] for r_id in json_list_links['result']['results']]
list_of_dictionaries_ = []
for elem in tqdm(all_elements):
    new_requ_ = elem
    if not new_requ_.endswith(".html"):
        new_requ_ = (new_requ_+".html")
    if "/erddap/info" not in new_requ_:
        new_requ_ = new_requ_.replace("/erddap/tabledap", "/erddap/info")
    if "index.html" in new_requ_:
        new_requ_ = new_requ_.replace("index.html", "index.json")
    if ".html" in new_requ_:
        new_requ_ = new_requ_.replace(".html", "/index.json")
    das_response = session.get(new_requ_)
    if das_response.reason == '404':
        print(f"URL not found: {elem}")
        continue
    json_obj_ = das_response.json()
    ds_profile_dictionary_ = parse_json(json_obj_)
    list_of_dictionaries_.append(ds_profile_dictionary_)
    time.sleep(1)

print(f"Number of das files: {list_of_dictionaries_.__len__()}")
pickle.dump(list_of_dictionaries_, open("./alldataset_das_dict_NEW.pkl", 'wb'))

## Step#2: 

Matching of overlapping dataset having same attributes

Input: 'alldataset_das_dict_NEW.pkl' file generated by '__download_erddap_metadata.py'
Output: 'df_attributal_matching_score.csv' 


In [ ]:
import pandas as pd
import numpy as np


# run __download_erddap_metadata.py
erdap_dict_ = pickle.load(open("../res/alldataset_das_dict_NEW.pkl", 'rb'))

list_of_das_variables_ = []
item_labels = []
for dset_ in erdap_dict_:
    item_labels.append(dset_['NC_GLOBAL']['title'][1])
    del [dset_['NC_GLOBAL']]
    list_of_das_variables_.append(set(dset_.keys()))

num_items =  len(list_of_das_variables_)
# Initialize a 50x50 matrix with zeros
matching_matrix = np.zeros((num_items, num_items), dtype=int)

lod = list_of_das_variables_
# Calculate matching scores
for i in range(num_items):
    for j in range(num_items):
        total_ = len(lod[i])
        match_count = len(lod[i].intersection(lod[j]))
        match_count = int(float(match_count/total_)*100)


        matching_matrix[i][j] = match_count


df_matching = pd.DataFrame(matching_matrix, index=item_labels, columns=item_labels)
df_matching.to_csv("./res/df_attributal_matching_score.csv")

## Step#3:
Calculating spatial matching score between datasets. the spatial window of 20KM is set.

Input: 'grid-ocean-lakes-20KM-EPSG-3857.shp' contains lakes map, generated from QGIS

Output: '_df_spatial_matching_score.csv' 


In [ ]:
import geopandas as gpd
import pickle

import geopandas as gpd
import numpy as np
from scipy.spatial import cKDTree
import os
from tqdm import tqdm
import pandas as pd


# this .shp file is created from QGIS which contains map of all lakes
grid_filename = "/res/grid-ocean-lakes-20KM-EPSG-3857.shp"
gdf_filename = grid_filename.replace(".shp","_gdf.pkl")
ckdtree_filename = grid_filename.replace(".shp","_ckdtree.pkl")
dataset_grid_dict_filename = grid_filename.replace(".shp", "_das_gridcenter_dict.pkl")

if not os.path.exists(gdf_filename):
    gdf_ = gpd.read_file(grid_filename)
    gdf_['centroid'] = gdf_.geometry.centroid
    target_crs = 'EPSG:4269'
    centroids_gdf = gdf_[['centroid']].copy()
    centroids_gdf.set_geometry('centroid', inplace=True)
    gdf_['centroid'] = centroids_gdf.to_crs(target_crs)
    gdf_['cen_x'] = gdf_['centroid'].x
    gdf_['cen_y'] = gdf_['centroid'].y

    nB = np.array(list(gdf_['centroid'].geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    pickle.dump(btree, open(ckdtree_filename, 'wb'))
    pickle.dump(gdf_, open(gdf_filename, 'wb'))
else:
    print("Loading GDF and ")
    gdf_ = pickle.load(open(gdf_filename, 'rb'))
    btree = pickle.load(open(ckdtree_filename, 'rb'))

dataset_grids_map = {} #dataset_idx = [ grid_centroids,.. ]
if not os.path.exists(dataset_grid_dict_filename):
    for root, dir, files in os.walk("../erddap data/"):
        pbar_ = tqdm(files)
        for fname in pbar_:
            pbar_.set_description(fname)
            if fname.lower().endswith(".csv"):
                fl_path_ = os.path.join(root, fname)
                df = pd.read_csv(fl_path_).dropna()
                if 'latitude' not in df.columns:
                    print(f"Skipping [{fname}]")
                    continue

                points = set(zip(df['latitude'].iloc[1:].astype('float64'), df['longitude'].iloc[1:].astype('float64')))
                points_ndarray = np.array(list(points))
                dist, idx = btree.query(points_ndarray, k=1)
                new_points = btree.data[idx]
                new_points_df_ = pd.DataFrame(new_points, columns=['cen_x', 'cen_y'])
                merged_df = gdf_.merge(new_points_df_, on=['cen_x', 'cen_y'], how='inner')
                matched_ids = set([int(x) for x in merged_df['id'].tolist()])
                dataset_grids_map[fname] = matched_ids

    pickle.dump(dataset_grids_map, open(dataset_grid_dict_filename, 'wb'))

else:
    dataset_grids_map = pickle.load(open(dataset_grid_dict_filename, 'rb'))


list_of_das_variables_ = []
item_labels = []
for d_name, grids_ in dataset_grids_map.items():
    item_labels.append(d_name)
    list_of_das_variables_.append(grids_)

num_items = len(list_of_das_variables_)
# Initialize a 50x50 matrix with zeros
matching_matrix = np.zeros((num_items, num_items), dtype=int)

lod = list_of_das_variables_
# Calculate matching scores
for i in tqdm(range(num_items)):
    for j in range(num_items):
        total_ = len(lod[i])
        match_count = len(set(lod[i]).intersection(set(lod[j])))
        match_count = int(float(match_count / total_) * 100)
        # if match_count != 100:
        #     match_count = 0
        matching_matrix[i][j] = match_count

df_matching = pd.DataFrame(matching_matrix, index=item_labels, columns=item_labels)
df_matching.to_csv(f"./res/{os.path.basename(grid_filename).replace('.shp','_df_spatial_matching_score.csv')}")

### Step#4:

Input: The data from erddap should been downloaded in the directory 'erddap data'.  Please refer to the Building Bridges directory in Google drive to copy the pre-downloaded datasets. 

This file traverse the timestamp of all datasets and calculate the overlapping dataset similarity
the window size of 24 hours has been set to check overlap
Output: date_df_temporal_matching_score

In [ ]:
dataset_date_map = {}
for root, dir, files in os.walk("../erddap data/"):
    pbar_ = tqdm(files)
    for fname in pbar_:
        pbar_.set_description(fname)
        if fname.lower().endswith(".csv"):
            fl_path_ = os.path.join(root, fname)
            df = pd.read_csv(fl_path_).dropna()
            if 'latitude' not in df.columns:
                print(f"Skipping [{fname}]")
                continue

            s_ = pd.to_datetime(df['time'].iloc[1:], format='%Y-%m-%dT%H:%M:%SZ').dt.strftime('%Y%m%d').astype(int)
            dataset_date_map[fname] = s_.unique()

list_of_das_variables_ = []
item_labels = []
for d_name, dates_list in dataset_date_map.items():
    item_labels.append(d_name)
    list_of_das_variables_.append(dates_list)

num_items = len(list_of_das_variables_)
# Initialize a 50x50 matrix with zeros
matching_matrix = np.zeros((num_items, num_items), dtype=int)

lod = list_of_das_variables_
# Calculate matching scores
for i in tqdm(range(num_items)):
    for j in range(num_items):
        total_ = len(lod[i])
        match_count = len(set(lod[i]).intersection(set(lod[j])))
        match_count = int(float(match_count / total_) * 100)

        matching_matrix[i][j] = match_count

df_matching = pd.DataFrame(matching_matrix, index=item_labels, columns=item_labels)
df_matching.to_csv(f"./res/date_df_temporal_matching_score.csv")

## Step#5:

Plotting 

![heatmap](res/screenshot_heatmap.png)

In [13]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import plotly.io as pio
pio.renderers.default = 'iframe'              # Good fallback for PyCharm


attr_score_ = pd.read_csv("../res/df_attributal_matching_score.csv", index_col=0)
spatial_score_ = pd.read_csv("../res/grid-ocean-lakes-20KM-EPSG-3857_df_spatial_matching_score.csv", index_col=0)
temporal_score_ = pd.read_csv("../res/date_df_temporal_matching_score.csv", index_col=0)

# Clean/rename indices to match
new_spatial_score_ = pd.DataFrame(columns=attr_score_.columns)
col_row_to_remove = []
rename_columns_ = {}
for i, row in attr_score_.iterrows():
    up_i = i.replace(",", " ").replace(":", "").replace("/", " ").replace("|", "") + ".csv"
    if up_i not in spatial_score_.index:
        col_row_to_remove.append(i)
    else:
        rename_columns_[i] = up_i
attr_score_.drop(columns=col_row_to_remove, inplace=True)
attr_score_.drop(col_row_to_remove, inplace=True)
attr_score_.rename(columns=rename_columns_, inplace=True)
attr_score_.rename(index=rename_columns_, inplace=True)

# Set thresholds manually
attr_threshold = 0
spatial_threshold = 0
temporal_threshold_ = 40
default_value = -1

# Apply threshold masks
mask1 = attr_score_ >= attr_threshold
mask2 = spatial_score_ >= spatial_threshold
mask3 = temporal_score_ >= temporal_threshold_
combined_mask = mask1 & mask2 & mask3

attr_score_df_ = attr_score_.where(combined_mask, default_value)
spatial_score_df_ = spatial_score_.where(combined_mask, default_value)
temporal_score_df_ = temporal_score_.where(combined_mask, default_value)

# Plot using plotly
colors = ['Greens', 'Blues', 'Oranges']

heatmap1 = go.Heatmap(
    z=attr_score_df_.values,
    x=list(attr_score_df_.columns),
    y=list(attr_score_df_.index),
    opacity=0.2,
    colorscale=colors[0],
    zmax=150,
    zmin=0,
    colorbar=dict(title='Attribute', orientation='h', len=0.33, x=0.1),
    showscale=True,
    customdata=np.dstack((spatial_score_df_, attr_score_df_, temporal_score_df_)),
    hovertemplate=(
        "Row: %{y}<br>"
        "Col: %{x}<br>"
        "Spatial: %{customdata[0]}<br>"
        "Temp: %{customdata[2]}<br>"
        "Attr: %{z}<br>"
        "<extra></extra>"
    )
)

heatmap2 = go.Heatmap(
    z=spatial_score_df_.values,
    x=list(spatial_score_df_.columns),
    y=list(spatial_score_df_.index),
    colorscale=colors[1],
    opacity=0.2,
    zmax=150,
    zmin=0,
    colorbar=dict(title='Spatial', orientation='h', len=0.33, x=0.8),
    showscale=True
)

heatmap3 = go.Heatmap(
    z=temporal_score_df_.values,
    x=list(temporal_score_df_.columns),
    y=list(temporal_score_df_.index),
    opacity=0.2,
    zmax=150,
    zmin=0,
    colorscale=colors[2],
    colorbar=dict(title='Temporal', orientation='h', len=0.33, x=0.46),
    showscale=True
)

fig = go.Figure(data=[heatmap3, heatmap2, heatmap1])
fig.update_layout(
    width=1800,
    height=1800,
    xaxis=dict(tickangle=90, tickfont=dict(size=4)),
    yaxis=dict(tickfont=dict(size=4)),
    title=dict(font=dict(size=20)),
    font=dict(size=10)
)

# Show the figure in notebook
fig.show()

## Problem Visualizing in Jupyter Notebook

If you are getting problem visualizing in notebook, run code below in .py file 

In [ ]:
attr_score_ = pd.read_csv("../res/df_attributal_matching_score.csv", index_col=0)
spatial_score_ = pd.read_csv("../res/grid-ocean-lakes-20KM-EPSG-3857_df_spatial_matching_score.csv", index_col=0)
temporal_score_ = pd.read_csv("../res/date_df_temporal_matching_score.csv", index_col=0)

new_spatial_score_ = pd.DataFrame(columns=attr_score_.columns)
col_row_to_remove = []
rename_columns_ = {}
for i, row in attr_score_.iterrows():
    up_i = i.replace(",", " ").replace(":", "").replace("/", " ").replace("|", "")
    up_i = up_i + ".csv"
    if up_i not in spatial_score_.index:
        col_row_to_remove.append(i)
    else:
        rename_columns_[i] = up_i

attr_score_.drop(columns=col_row_to_remove, inplace=True)
attr_score_.drop(col_row_to_remove, inplace=True)
attr_score_.rename(columns=rename_columns_, inplace=True)
attr_score_.rename(index=rename_columns_, inplace=True)


attr_min__ = 0
attr_max__ = max(attr_score_.max())
spatial_min__ = 0
spatial_max__ = max(spatial_score_.max())
temporal_min__ = 0
temporal_max__ = min(temporal_score_.max())


# Initialize the Dash app
app = dash.Dash(__name__)
app.title = "Interactive Heatmap with Thresholds"

# Define the layout
app.layout = html.Div([
    html.H1("Interactive Heatmap with Threshold Filters", style={'textAlign': 'center'}),

    html.Div([
        html.Div([
            html.Label("Temporal Threshold:", style={'font-weight': 'bold'}),
            dcc.Slider(
                id='temporal-threshold',
                min=temporal_min__,
                max=temporal_max__,
                step=1,
                value=40,  # Default value
                marks={i: str(i) for i in range(0, 101, 10)},
                tooltip={"placement": "bottom", "always_visible": True},
            ),
        ], style={'width': '45%', 'display': 'inline-block', 'padding': '0 20'}),

        html.Div([
            html.Label("Attributes Threshold:", style={'font-weight': 'bold'}),
            dcc.Slider(
                id='attr-threshold',
                min=attr_min__,
                max=attr_max__,
                step=1,
                value=0,  # Default value
                marks={i: str(i) for i in range(0, 101, 10)},
                tooltip={"placement": "bottom", "always_visible": True},
            ),
        ], style={'width': '45%', 'display': 'inline-block', 'padding': '0 20'}),

        html.Div([
            html.Label("Spatial Threshold:", style={'font-weight': 'bold'}),
            dcc.Slider(
                id='spatial-threshold',
                min=spatial_min__,
                max=spatial_max__,
                step=1,
                value=0,  # Default value
                marks={i: str(i) for i in range(0, 101, 10)},
                tooltip={"placement": "bottom", "always_visible": True},
            ),
        ], style={'width': '45%', 'display': 'inline-block', 'padding': '0 20'}),
    ], style={'padding': '40px'}),

    dcc.Graph(id='heatmap')
])


# Define the callback to update the heatmap
@app.callback(
    Output('heatmap', 'figure'),
    [Input('attr-threshold', 'value'),
     Input('spatial-threshold', 'value'),
     Input('temporal-threshold', 'value')]
)


def update_heatmap(attr_threshold, spatial_threshold, temporal_threshold_):
    # Create a mask based on the thresholds
    attr_score_df_, spatial_score_df_, temporal_score_df_ = attr_score_.copy(), spatial_score_.copy(), temporal_score_.copy()
    mask1 = attr_score_df_ >= attr_threshold
    mask2 = spatial_score_df_ >= spatial_threshold
    mask3 = temporal_score_df_ >= temporal_threshold_
    # print(f"Attr: {attr_threshold}  - Spatial: {spatial_threshold} - Temp: {temporal_threshold_}")
    combined_mask = mask1 & mask2 & mask3
    default_value  = -1
    attr_score_df_.where(combined_mask, default_value, inplace=True)
    spatial_score_df_.where(combined_mask, default_value,  inplace=True)
    temporal_score_df_.where(combined_mask, default_value, inplace=True)

    #validation
    flag_ = False
    for col in attr_score_df_.columns:
        for idx in attr_score_df_.index.values:
            if (attr_score_df_.loc[idx][col] != default_value) and (attr_score_df_.loc[idx][col] < attr_threshold):
                flag_ = True
                print(f"ATTR [{attr_score_df_.loc[idx][col]}  < {attr_threshold}]")
                break
            if (spatial_score_df_.loc[idx][col] != default_value) and (spatial_score_df_.loc[idx][col] < spatial_threshold):
                print(f"SPAT [{spatial_score_df_.loc[idx][col]}  < {spatial_threshold}]")
                flag_ = True
                break
            if (temporal_score_df_.loc[idx][col] != default_value) and (temporal_score_df_.loc[idx][col] < temporal_threshold_):
                print(f"TEMP [{temporal_score_df_.loc[idx][col]} < {temporal_threshold_}]")
                flag_ = True
                break
        if flag_:
            break
    if flag_:
        print("Something is wrong.....!")


    colors = ['Greens', 'Blues', 'Oranges']
    # colors = ['Purp', 'Oryel','Reds']
    # Create the heatmap
    heatmap1 = go.Heatmap(
        z=attr_score_df_.values,
        x=list(attr_score_df_.columns),
        y=list(attr_score_df_.index),
        opacity=0.2,
        colorscale=colors[0],
        zmax=150, zauto=False, zmin=0,
        colorbar=dict(title='Attribute', orientation='h', len=0.33, x=0.1),
        showscale=True,
        hovertemplate=(
            "Row: %{y}<br>"  # Display y-coordinate as row index
            "Col: %{x}<br>"  # Display x-coordinate as column index
            "Spatial: %{customdata[0]}<br>"  # Value from df1 based on x,y
            "Temp: %{customdata[2]}<br>"  # Value from df3 based on x,y
            "Attr: %{z}<br>"  # Value from df2 based on x,y
            "<extra></extra>"  # Removes extra trace info from hover
        ),
        customdata=np.dstack((spatial_score_df_, attr_score_df_, temporal_score_df_)),

    )
    heatmap2 = go.Heatmap(
        z=spatial_score_df_.values,
        x=list(spatial_score_df_.columns),
        y=list(spatial_score_df_.index),
        colorscale=colors[1],
        opacity=0.2,zmax=150, zauto=False, zmin=0,
        colorbar=dict(
            title='Spatial',
            x=0.8,  # Position at the bottom center
            orientation='h', len=0.33
        ),
        showscale=True
    )

    # Create the heatmap
    heatmap3 = go.Heatmap(
        z=temporal_score_df_.values,
        x=list(temporal_score_df_.columns),
        y=list(temporal_score_df_.index),
        opacity=0.2,zmax=150, zauto=False, zmin=0,
        colorscale=colors[2],
        colorbar=dict(
            title='Temporal',
            titlefont=dict(size=12),
            tickfont=dict(size=10),
            x=0.46,  # Position at the bottom center
            orientation='h', len=0.33
        ),
        showscale=True
    )

    fig = go.Figure(data=[heatmap3, heatmap2, heatmap1])
    fig.update_layout(
        width=1800,  # Increase width
        height=1800,  # Increase height
        xaxis=dict(tickangle=90, tickfont=dict(size=4)),
        yaxis=dict(tickfont=dict(size=4)),
        title=dict(font=dict(size=20)),
        font=dict(size=10)

    )
    return fig


app.run_server(debug=True)